In [17]:
import joblib
from statistics import mean
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import pandas as pd
import re
import newspaper
import requests
from pprint import pprint

In [18]:

test_set = pd.read_csv('articles.csv', encoding='utf-8')
model = joblib.load('vote_model.joblib')

# url = ('http://newsapi.org/v2/top-headlines?country=us&pageSize=100&category=business&apiKey=f4767a5c003944e5bbe9b97170bb65c0')
# response = requests.get(url)
# data = response.json()


# if not data['status'] == 'ok':
#     print("oopsie")
#     exit()

def clean(text):

    text = re.sub('[\n\t,]', ' ', text)
    text = text.replace('Advertisement', ' ')

    article = []
    article.append(text)
    articles = pd.Series(article)

    stemmer = SnowballStemmer('english')
    words = stopwords.words("english")

    cleaned = articles.apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

    return cleaned


# recent = pd.Series(dtype=str)
# for article in data['articles']:
#     if not article['content'] == None:
#         recent = recent.append(pd.Series([article['content']], dtype=str))
    
display(recent)

recent = test_set.loc[test_set['year'] == 2017]
recent = recent.sample(frac=0.05)
recent = recent.content

10434    When Octavia Spencer first read the script for...
3502     ’Read the relevant court documents, made avail...
4123     Washington (CNN) House Speaker Paul Ryan and h...
2143     ’   The man authorities believe shot 13 people...
774      Dale Russakoff | The New York Times Magazine, ...
                               ...                        
9682     DALLAS  —   Mississippi State lists its starti...
10409    It took an explosion and 13 pounds of iron to ...
6178     Who is doing what? A firm in Abu Dhabi has flo...
6498     British MPs lined up on Monday to pour scorn o...
6685     How close the Senate Republicans are to repeal...
Name: content, Length: 977, dtype: object

In [19]:
def sentiment(text):
    X = clean(text)
    conf_score = model.transform(X)[0]
    scores = []
    all_classes = []
    for i in range(int(len(conf_score)/2)):
        all_classes.append(conf_score[2*i+1])
    score = mean(all_classes)*4-2
    print(score)
#     if score < -0.5:
#         if score > -0.55:
#             scores = ['left', 0]
#         elif score > -0.7:
#             scores = ['left', 0.3]
#         elif score > -0.85:
#             scores = ['left', 0.6]
#         else:
#             scores = ['left', 1]
#     else:
#         if score < -0.45:
#             scores = ['right', 0]
#         elif score < -0.3:
#             scores = ['right', 0.3]
#         elif score < -0.15:
#             scores = ['right', 0.6]
#         else:
#             scores = ['right', 1]
    if score < -0.9:
        if score > -1.05:
            scores = ['left', 0]
        elif score > -1.4:
            scores = ['left', 0.3]
        elif score > -1.6:
            scores = ['left', 0.6]
        else:
            scores = ['left', 1]
    else:
        if score < -0.75:
            scores = ['right', 0]
        elif score < -0.3:
            scores = ['right', 0.3]
        elif score < 0.5:
            scores = ['right', 0.6]
        else:
            scores = ['right', 1]
    return scores

In [20]:
sentiments = []

for article in recent.tolist():
    if (type(article) == str):
        sent = sentiment(article)
        if len(sent) == 2:
            sentiments.append(sent)

neutral = 0
slight_left = 0
slight_right = 0
left = 0
right = 0
far_left = 0
far_right = 0

for s in sentiments:
    print(s)
    if s[1] == 0:
        neutral+=1
    elif s[0] == 'left':
        if s[1] == 0.3:
            slight_left+=1
        elif s[1] == 0.6:
            left+=1
        else:
            far_left+=1
    else:
        if s[1] == 0.3:
            slight_right+=1
        elif s[1] == 0.6:
            right+=1
        else:
            far_right+=1
        

print(f'     neutral: {neutral}\n')        
        
print(f' slight left: {slight_left}')
print(f'        left: {left}')
print(f'    far left: {far_left}')
print(f'  total left: {slight_left+left+far_left}\n')

print(f'slight right: {slight_right}')
print(f'       right: {right}')
print(f'   far right: {far_right}')
print(f'  total right: {slight_right+right+far_right}\n')



0.7055309072407794
-0.2681860503946685
-1.181316420380516
-1.4576633663274903
0.6214009452769318
-1.5639109396182667
-1.2903975432513142
-1.3226020191768475
-1.3264056973691263
-1.6437265990054173
-0.10642261624618854
-1.781042815165408
1.1381003861526091
-1.4896499204430174
-0.39357642724824915
-1.6852105120252232
-1.2066478587091816
-1.5455868193057638
-1.2474077211080132
-1.2404128670897072
-1.263886958373067
-0.03140820445561321
-0.9755932280927506
-1.1155734123178147
-1.845798227491628
-0.3572811586322566
-0.1108427444102118
-0.5599401596794176
-1.536547567062129
-1.5720414233925761
0.33394822228474785
-1.3714583613830698
-0.44697138117788193
-0.33334902706829106
-0.5577027238561056
-1.3592018280968108
-1.7675042775812215
0.29050634223405103
-1.5344904147466691
-0.7595441933803477
-1.4089484450134053
-1.4060883162099929
-0.6289176576159656
-0.6187282931100013
-1.0740324308886788
-1.4370921554155696
-0.37111983396065606
-0.2943472210459157
-1.435936920820466
-1.2074118989952267
0.0